In [ ]:
from unittest import result
import ray
from ray import tune
from ray.rllib.algorithms.random_agent import RandomAgentConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune import register_env

import rllib_setup


def train(jobs, machine):
    env_name = "VJS_" + str(machine) + "_" + str(jobs)
    alg_name = "Random"
    register_env(
        env_name,
        lambda config: rllib_setup.get_env_continuous(jobs, machine),
    )
    test_env = rllib_setup.get_env_continuous(jobs, machine)

    def policies(agent_ids):
        obs_space = test_env.observation_space
        act_space = test_env.action_space
        return {
            str(i): (
                None,
                obs_space,
                act_space,
                {}
                # config=config.overrides(agent_id=int(i[8:])),
            )
            for i in agent_ids
        }

    config = (
        RandomAgentConfig()
        .rollouts(
            num_rollout_workers=10,
            rollout_fragment_length=30,
        )
        .resources(num_gpus=1)
        .multi_agent(
            policies=policies(test_env._agent_ids),
            policy_mapping_fn=lambda agent_id, episode, **kwargs: str(agent_id),
        )
        .environment(env=env_name, disable_env_checking=True)
    )
    config.batch_mode = "complete_episodes"
    print(config.to_dict())
    # Build a Algorithm object from the config and run one training iteration.
    # algo = config.build(env=env_name)

    return config




In [ ]:

ray.init()


In [4]:
config = train(60, 12)

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 1, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'num_learner_workers': 0, 'num_gpus_per_learner_worker': 0, 'num_cpus_per_learner_worker': 1, 'local_gpu_idx': 0, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': 'VJS_12_60', 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': True, 'is_atari': None, 'auto_wrap_old_gym_envs': True, 'num_

(raylet) [2023-10-04 15:19:59,275 E 1584314 1584326] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-10-04_15-18-37_078976_1583807 is over 95% full, available space: 58261504; capacity: 1967315451904. Object creation will fail if spilling is required.


In [6]:
algo = config.build()

/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/rllib/algorithms/algorithm.py:442: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/yuan/ResMan/man/lib/python3.9/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future 

In [7]:
algo.evaluate()

AttributeError: 'NoneType' object has no attribute 'local_worker'

(raylet) [2023-10-04 15:21:29,365 E 1584314 1584326] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-10-04_15-18-37_078976_1583807 is over 95% full, available space: 58105856; capacity: 1967315451904. Object creation will fail if spilling is required.
(raylet) [2023-10-04 15:21:39,373 E 1584314 1584326] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-10-04_15-18-37_078976_1583807 is over 95% full, available space: 58077184; capacity: 1967315451904. Object creation will fail if spilling is required.
(raylet) [2023-10-04 15:21:49,383 E 1584314 1584326] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-10-04_15-18-37_078976_1583807 is over 95% full, available space: 57810944; capacity: 1967315451904. Object creation will fail if spilling is required.
(raylet) [2023-10-04 15:21:59,393 E 1584314 1584326] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-10-04_15-18-37_078976_1583807 is over 95% full, available space: 57778176; capacity: 1967315451904.